In [1]:
from tools.io import read_json, write_json
import numpy as np
from pathlib import Path

In [6]:
droot = Path("dataset")/"p2"

In [5]:
table = {}

for ti in droot.iterdir():
    if ti.is_dir():
        org_img = [_ for _ in ti.glob("*.jpg") if 'aug' not in _.stem]
        idxs = np.random.permutation(np.arange(len(org_img)))
        train_to = int(len(org_img)*0.8)
        train_idx = idxs[:train_to]
        real_train = int(len(train_idx)*0.9)
        real_train_idx = train_idx[:real_train]
        valid_idx = train_idx[real_train:]
        to_test = idxs[train_to:]
        table[ti.parts[-1]] = {
            'train':sorted([str(org_img[i]) for i in real_train_idx], key=lambda x:int(Path(x).stem)),
            'valid':sorted([str(org_img[i]) for i in valid_idx], key=lambda x:int(Path(x).stem)),
            'test':sorted([str(org_img[i]) for i in to_test], key=lambda x:int(Path(x).stem))
        }
write_json(table, Path("dataset")/"train_valid_test"/"baseline.json")

In [9]:
aug_table = {}
for ti in table:
    aug_table[ti] = table[ti].copy()
    if ti == "Type0":
        continue

    aug_train = []
    for i in table[ti]['train']:
        imid = Path(i).stem
        aug_train += [str(_) for _ in (droot/ti).glob(f"aug*_{imid}.jpg") ]

    aug_table[ti]['train'] = sorted(
        aug_train + aug_table[ti]['train'],
        key=lambda x:int(Path(x).stem.split("_")[-1])
    )
write_json(aug_table, Path("dataset")/"train_valid_test"/"aug.json")

In [ ]:
down_table = {}
for ti in table:
    print(ti)
    down_table[ti] = {}
    if ti == "Type1":
        down_table[ti] = aug_table[ti].copy()
    elif ti == "Type0":
        down_idx_train = np.random.permutation(np.arange(len(table[ti]['train'])))[:500]
        down_idx_val = np.random.permutation(np.arange(len(table[ti]['valid'])))[:55]
        down_table[ti]['train'] = sorted(
            [table[ti]['train'][i] for i in down_idx_train], 
            key=lambda x:int(Path(x).stem)
        )
        down_table[ti]['valid'] = sorted(
            [table[ti]['valid'][i] for i in down_idx_val], 
            key=lambda x:int(Path(x).stem)
        )
    else:
        down_table[ti] = table[ti].copy()
print(len(down_table["Type0"]['train']))       
write_json(down_table, Path("dataset")/"train_valid_test"/"down.json")